In [1]:
import lxml.etree as ET
from tqdm import tqdm

root = ET.parse('/Users/williamdiakite/Desktop/cbc.xml/conbavil.xml').getroot()

namespaceX = { 
    'cbc': 'http://conbavil.fr/namespace',
    'xml': 'http://www.w3.org/XML/1998/namespace',
    'fonc': "http://conbavil.fr/fn"
}

namespace = { 
    None: 'http://conbavil.fr/namespace',
    'fonc': "http://conbavil.fr/fn",
    'xml': 'http://www.w3.org/XML/1998/namespace'
}

cbc = '{http://conbavil.fr/namespace}'
xml = '{http://www.w3.org/XML/1998/namespace}'

db = ET.Element(cbc + 'facets', nsmap=namespace)

In [2]:
uniques = {}
facets = ['commune', 'departement', 'region']

for f in facets:
    values = []
    for v in root.xpath('.//cbc:' + f, namespaces=namespaceX):
        t = v.text
        if isinstance(t, str):
            values.append(t)
    values = list(set(values))

    uniques[f] = values


for k in uniques:
    print('creating facet', k, '\n')
    facet = ET.SubElement(db, cbc + k + 's')

    for v in tqdm(uniques[k]):
        f = ET.SubElement(facet, cbc + k)
        f.set('text', v)

        deliberations = ET.SubElement(f, cbc + 'deliberations')
        affairs = ET.SubElement(f, cbc + 'affairs')
        meetings = ET.SubElement(f, cbc + 'meetings')

        # deliberations
        exp = './/cbc:deliberation[cbc:localisation/cbc:%s/text() = "%s"]/@xml:id' % (k, v)
        # print(exp, '\n')
        for xml_id in root.xpath(exp, namespaces=namespaceX):
            d = ET.SubElement(deliberations, cbc + 'deliberation')
            d.attrib[xml + 'id'] = xml_id

        # affairs
        exp = './/cbc:affair[cbc:localisation/cbc:%s/text() = "%s"]/@xml:id' % (k, v)
        # print(exp, '\n')
        for xmlId in root.xpath(exp, namespaces=namespaceX):
            d = ET.SubElement(affairs, cbc + 'affair')
            d.attrib[xml + 'id'] = xml_id

        # meetings
        exp = './/cbc:meeting[cbc:deliberations/cbc:deliberation/cbc:localisation/cbc:%s/text() = "%s"]/@xml:id' % (k, v)
        # print(exp, '\n')
        for xmlId in root.xpath(exp, namespaces=namespaceX):
            d = ET.SubElement(meetings, cbc + 'meeting')
            d.attrib[xml + 'id'] = xml_id

        

creating facet commune 



100%|██████████| 5797/5797 [29:58<00:00,  3.22it/s]


creating facet departement 



100%|██████████| 608/608 [03:08<00:00,  3.22it/s]


creating facet region 



100%|██████████| 77/77 [00:25<00:00,  2.98it/s]


In [3]:
uniques = {}
facets = ['buildingType', 'projectGenre', 'administrativeObject']

for f in facets:
    values = []
    for v in root.xpath('.//cbc:category[@type="%s"]' % f, namespaces=namespaceX):
        t = v.text
        if isinstance(t, str):
            values.append(t)
    values = list(set(values))

    uniques[f] = values

categories = ET.SubElement(db, cbc + 'categories')

for k in uniques:
    print('creating facet', k, '\n')

    for v in tqdm(uniques[k]):
        f = ET.SubElement(categories, cbc + 'categories')
        f.set('text', v)
        f.set('type', k)

        deliberations = ET.SubElement(f, cbc + 'deliberations')
        affairs = ET.SubElement(f, cbc + 'affairs')
        meetings = ET.SubElement(f, cbc + 'meetings')

        # deliberations
        exp = './/cbc:deliberation[cbc:categories/cbc:category[@type = "%s"]/text() = "%s"]/@xml:id' % (k, v)
        # print(exp, '\n')
        for xml_id in root.xpath(exp, namespaces=namespaceX):
            d = ET.SubElement(deliberations, cbc + 'deliberation')
            d.attrib[xml + 'id'] = xml_id
       
        # affairs
        exp = './/cbc:affair[cbc:categories/cbc:category[@type = "%s"]/text() = "%s"]/@xml:id' % (k, v)
        # print(exp, '\n')
        for xml_id in root.xpath(exp, namespaces=namespaceX):
            d = ET.SubElement(affairs, cbc + 'affair')
            d.attrib[xml + 'id'] = xml_id
        
        # meetings
        exp = './/cbc:meeting[cbc:deliberations/cbc:deliberation/cbc:categories/cbc:category[@type = "%s"]/text() = "%s"]/@xml:id' % (k, v)
        # print(exp, '\n')
        for xml_id in root.xpath(exp, namespaces=namespaceX):
            d = ET.SubElement(meetings, cbc + 'meeting')
            d.attrib[xml + 'id'] = xml_id


creating facet buildingType 



100%|██████████| 444/444 [03:36<00:00,  2.05it/s]


creating facet projectGenre 



100%|██████████| 115/115 [00:52<00:00,  2.20it/s]


creating facet administrativeObject 



100%|██████████| 1077/1077 [08:15<00:00,  2.17it/s]


In [4]:
ET.ElementTree(db).write('../facets.xml', pretty_print=True, encoding='UTF-8')